In [1]:
from pulp import *
import pandas as pd
import numpy as np
import math

initial_loading = pd.read_csv('initial_loading.csv')
product_specs = pd.read_csv('product_specs.csv')
tam_margins = pd.read_csv('tam_and_margin.csv')
workstation_metrics = pd.read_csv('workstation_metrics.csv').T.fillna(0)

workstation_metrics.columns = workstation_metrics.iloc[0]
workstation_metrics = workstation_metrics.drop(workstation_metrics.index[0])

In [2]:
# def create_abs_difference(model, x, y, name="abs_diff"):
#     """
#     Creates variables and constraints to represent |x - y| in a PuLP model.
    
#     Parameters:
#     model: PuLP LpProblem
#     x: PuLP variable or number representing first value
#     y: PuLP variable or number representing second value
#     name: Base name for the new variables (default: "abs_diff")
    
#     Returns:
#     abs_diff: PuLP variable representing |x - y|
#     """
#     # Create a variable for the absolute difference
#     abs_diff = LpVariable(f"{name}", lowBound=0)
    
#     # Create positive and negative part variables
#     pos_part = LpVariable(f"{name}_pos", lowBound=0)
#     neg_part = LpVariable(f"{name}_neg", lowBound=0)
    
#     # Add constraints
#     # x - y = pos_part - neg_part
#     model += x - y == pos_part - neg_part, f"{name}_decomposition"
    
#     # abs_diff = pos_part + neg_part
#     model += abs_diff == pos_part + neg_part, f"{name}_definition"
    
#     return abs_diff

In [8]:
def tool_requirement(loading, minute_load, utilization):
    return loading * minute_load / (7 * 24 * 60 * utilization)
    
def gb_output_coef(node_gb_per_wafer, node_yield):
    return 13 * node_gb_per_wafer * node_yield / 1e9 # in billions, hence /1e9

def calculate_gb_output(loading_dict, quarter, specs_df):
    """Calculate GB output for a given quarter's loading"""
    total_gb = 0
    for node_idx, loading in loading_dict.items():
        gb_per_wafer = specs_df[specs_df['quarter'] == quarter][f'node{node_idx}_gb_per_wafer'].values[0]
        yield_rate = specs_df[specs_df['quarter'] == quarter][f'node{node_idx}_yield'].values[0]
        total_gb += loading * gb_output_coef(gb_per_wafer, yield_rate)
    return total_gb

def optimize_loading(min_node_3_wafers=0):
    """
    Optimize loading to minimize total wafers while meeting TAM with integer constraints
    
    Args:
        min_wafers: Minimum number of wafers required per node per quarter
    """
    results = {"loading": {}, "tools": {}, "capex": {}, "revenue": {}}
    quarters = product_specs['quarter'].unique()
    
    # Get initial loading
    current_loading = {
        node_idx: initial_loading[f'node{node_idx}_loading'].values[0]
        for node_idx in range(1, 4)
    }
    current_tool_count = {
        ws: int(row['initial_tool_count']) for ws, row in workstation_metrics.iterrows()
    }
    
    results["loading"][quarters[0]] = current_loading.copy()
    results["tools"][quarters[0]] = current_tool_count.copy()
    results["capex"][quarters[0]] = 0

    results["revenue"][quarters[0]] = calculate_gb_output(current_loading, quarters[0], product_specs) * 0.002 * 1e9
    
    # Optimize each quarter
    for i in range(1, len(quarters)):
        quarter = quarters[i]
        prev_quarter = quarters[i-1]
        
        prob = LpProblem(f"Profit_Maximization_{quarter}", LpMaximize)
        
        # Decision variables - specified as integers
        node1 = LpVariable(f"node1_{quarter}", 0, None, LpInteger)
        node2 = LpVariable(f"node2_{quarter}", 0, None, LpInteger)
        node3 = LpVariable(f"node3_{quarter}", min_node_3_wafers, None, LpInteger)
        
        nodes = {
            1: node1,
            2: node2,
            3: node3
        }

        workstations = {
            ws: LpVariable(f"ws{ws}_{quarter}", tool_count, None, LpInteger)for ws, tool_count in current_tool_count.items()
        }

        capex = LpVariable(f"capex_{quarter}", 0, None, LpInteger)
        
        # Target TAM
        target_tam = tam_margins[tam_margins['quarter'] == quarter]['tam'].values[0]
        
        # Calculate GB output coefficients
        quarter_specs = product_specs[product_specs['quarter'] == quarter]
        coefs = {
            node_idx: gb_output_coef(
                quarter_specs[f'node{node_idx}_gb_per_wafer'].values[0],
                quarter_specs[f'node{node_idx}_yield'].values[0]
            ) 
            for node_idx in nodes.keys()
        }

        gb_output = sum([coefs[node_idx] * node for node_idx, node in nodes.items()])
        revenue = gb_output * tam_margins[tam_margins['quarter'] == quarter]['contribution_margin'].values[0] * 1e9

        # for node_idx, node in nodes.items():
        #     for ws, ws_metrics in workstation_metrics.iterrows():
        #         prob += workstations[ws] >= tool_requirement(
        #             node, 
        #             ws_metrics[f"node_{node_idx}_minute_load"], 
        #             ws_metrics[f"utilization"]
        #         ), f"WS_min_{ws}_node_{node_idx}_{quarter}"


        capex = sum((workstations[ws] - current_tool_count[ws]) * workstation_metrics['capex_per_tool'][ws] for ws in workstations)

        prob += revenue - capex, f"Maximize_Profit_{quarter}"

        # Constraints
        # 1. Quarter to quarter loading change constraint (±2500)
        for node_idx, node in nodes.items():
            prob += node >= results["loading"][prev_quarter][node_idx] - 2500, f"Node{node_idx}_min_change_{quarter}"
            prob += node <= results["loading"][prev_quarter][node_idx] + 2500, f"Node{node_idx}_max_change_{quarter}"
        
        # 2. TAM constraints (±2B from target)
        prob += gb_output >= target_tam - 0, f"Min_TAM_{quarter}"
        prob += gb_output <= target_tam + 2, f"Max_TAM_{quarter}"
        
        # create_abs_difference(prob, target_tam, gb_output, f"abs_tams_diff_{quarter}")

        # 3. Workstation tool count
        
        for ws, ws_metrics in workstation_metrics.iterrows():
            tool_requirements = [
                tool_requirement(
                    node, 
                    ws_metrics[f"node_{node_idx}_minute_load"], 
                    ws_metrics[f"utilization"],
                ) for node_idx, node in nodes.items()
            ]

            prob += workstations[ws] >= sum(tool_requirements), f"WS_min_{ws}_{quarter}"
        
        # Solve
        status = prob.solve(getSolver('PULP_CBC_CMD'))
        
        if status != 1:
            print(f"\nWarning: Could not find optimal solution for {quarter}")
            print(f"Status code: {status}")
            print("Solver messages:")
            print(prob.solver.problem.status())
            continue

        current_tool_count = {
            ws: tool_count.value() for ws, tool_count in workstations.items()
        }
            
        results["loading"][quarter] = {
            node_idx: math.floor(node.value()) for node_idx, node in nodes.items()
        }

        results["tools"][quarter] = current_tool_count
        results["revenue"][quarter] = revenue.value()
        results["capex"][quarter] = capex.value()
        
        # Print quarter results
        total_wafers = sum(results["loading"][quarter].values())
        gb_output_val = calculate_gb_output(results["loading"][quarter], quarter, product_specs)
        print(f"\n{quarter}:")
        print(f"Total wafers: {total_wafers}")
        print(f"GB output: {gb_output_val:.2f}B vs target {target_tam:.2f}B")
        for node in range(1, 4):
            print(f"{node}: {results['loading'][quarter][node]} wafers")
            if i > 0:
                change = results["loading"][quarter][node] - results["loading"][prev_quarter][node]
                print(f"  Change from prev quarter: {change:+d}")
    
    return results

def validate_solution(results):
    """Validate that solution meets all constraints with detailed output"""
    quarters = list(results['loading'].keys())
    violations = []
    
    for i in range(len(quarters)):
        quarter = quarters[i]
        
        # Check integer and non-negative loading
        for node in range(1, 4):
            if not isinstance(results['loading'][quarter][node], int):
                violations.append(f"{quarter}: {node} has non-integer loading: {results['loading'][quarter][node]}")
            if results['loading'][quarter][node] < 0:
                violations.append(f"{quarter}: {node} has negative loading: {results['loading'][quarter][node]}")
        
        # Check quarter-to-quarter change
        if i > 0:
            prev_quarter = quarters[i-1]
            for node in range(1, 4):
                change = results['loading'][quarter][node] - results['loading'][prev_quarter][node]
                if abs(change) > 2500:
                    violations.append(
                        f"{quarter}: {node} change exceeds limit: {change:+d} "
                        f"(from {results['loading'][prev_quarter][node]} to {results['loading'][quarter][node]})"
                    )
        
        # Check TAM constraints
        gb_output = calculate_gb_output(results['loading'][quarter], quarter, product_specs)
        target_tam = tam_margins[tam_margins['quarter'] == quarter]['tam'].values[0]
        if abs(gb_output - target_tam) > 2:
            violations.append(
                f"{quarter}: TAM deviation exceeds 2B: output={gb_output:.2f}B, "
                f"target={target_tam:.2f}B, diff={gb_output-target_tam:+.2f}B"
            )
    
    if violations:
        print("\nConstraint Violations:")
        for v in violations:
            print(f"- {v}")
        return False
    
    print("\nAll constraints satisfied:")
    print(f"- Integer loading values ✓")
    print(f"- Non-negative loading ✓")
    print(f"- Quarter-to-quarter changes within ±2500 ✓")
    print(f"- TAM requirements met within ±2B ✓")
    return True

In [9]:
results = optimize_loading(min_node_3_wafers = 1000)
    
# Validate solution
if validate_solution(results):
    print("\nAll constraints satisfied")
else:
    print("\nSome constraints violated")

# Convert results to DataFrame for easy viewing
# qn1_part_b_results_df = pd.DataFrame(results).T
# qn1_part_b_results_df.index.name = 'quarter'

# qn1_part_b_results_df


Q2'26:
Total wafers: 20733
GB output: 27.56B vs target 27.40B
1: 14500 wafers
  Change from prev quarter: +2500
2: 5007 wafers
  Change from prev quarter: +7
3: 1226 wafers
  Change from prev quarter: +226

Q3'26:
Total wafers: 24150
GB output: 35.02B vs target 34.90B
1: 15718 wafers
  Change from prev quarter: +1218
2: 7432 wafers
  Change from prev quarter: +2425
3: 1000 wafers
  Change from prev quarter: -226

Q4'26:
Total wafers: 25552
GB output: 39.03B vs target 39.00B
1: 15107 wafers
  Change from prev quarter: -611
2: 9300 wafers
  Change from prev quarter: +1868
3: 1145 wafers
  Change from prev quarter: +145

Q1'27:
Total wafers: 27921
GB output: 44.74B vs target 44.70B
1: 14775 wafers
  Change from prev quarter: -332
2: 10998 wafers
  Change from prev quarter: +1698
3: 2148 wafers
  Change from prev quarter: +1003

Q2'27:
Total wafers: 30094
GB output: 51.51B vs target 51.50B
1: 13921 wafers
  Change from prev quarter: -854
2: 13496 wafers
  Change from prev quarter: +2498
3

In [10]:
results['capex']

{"Q1'26": 0,
 "Q2'26": 39000000.0,
 "Q3'26": 60300000.0,
 "Q4'26": 28000000.0,
 "Q1'27": 50300000.0,
 "Q2'27": 48700000.0,
 "Q3'27": 6000000.0,
 "Q4'27": 5100000.0}

In [11]:
pd.DataFrame(results["loading"]).to_csv('part_b_ans.csv')
pd.DataFrame(results["loading"]).T

PermissionError: [Errno 13] Permission denied: 'part_b_ans.csv'

In [12]:
pd.DataFrame(results["tools"]).T

,A,B,C,D,E,F,G,H,I,J
Q1'26,10.0,18.0,5.0,11.0,15.0,2.0,23.0,3.0,4.0,1.0
Q2'26,11.0,20.0,6.0,13.0,16.0,2.0,28.0,4.0,4.0,1.0
Q3'26,13.0,23.0,7.0,15.0,20.0,3.0,29.0,4.0,6.0,1.0
Q4'26,13.0,25.0,7.0,16.0,22.0,3.0,29.0,4.0,8.0,1.0
Q1'27,15.0,29.0,8.0,16.0,24.0,4.0,30.0,4.0,9.0,1.0
Q2'27,16.0,32.0,9.0,17.0,27.0,5.0,30.0,4.0,11.0,1.0
Q3'27,16.0,33.0,9.0,17.0,27.0,5.0,30.0,4.0,11.0,1.0
Q4'27,16.0,33.0,9.0,18.0,27.0,5.0,31.0,4.0,11.0,1.0


In [13]:
results["capex"]

{"Q1'26": 0,
 "Q2'26": 39000000.0,
 "Q3'26": 60300000.0,
 "Q4'26": 28000000.0,
 "Q1'27": 50300000.0,
 "Q2'27": 48700000.0,
 "Q3'27": 6000000.0,
 "Q4'27": 5100000.0}

In [14]:
results["revenue"]

{"Q1'26": 43680000.00000001,
 "Q2'26": 55110016.0,
 "Q3'26": 70042024.0,
 "Q4'26": 78056186.0,
 "Q1'27": 89482380.0,
 "Q2'27": 103026079.0,
 "Q3'27": 105497821.0,
 "Q4'27": 107455530.0}

In [15]:
profits = [results['revenue'][quarter] - results['capex'][quarter] for quarter in results['capex'].keys()]
net_profits = sum(profits)
print(f"{net_profits:,}")

414,950,036.0
